# Recommender Systems (and their implementations in Python)

Recommender systems are among the most popular applications of data science today. They are used to predict the "rating" or "preference" that a user would give to an item. Almost every major tech company has applied them in some form or the other: Amazon uses it to suggest products to customers, YouTube uses it to decide which video to play next on autoplay, and Facebook uses it to recommend pages to like and people to follow. What's more, for some companies -think Netflix and Spotify-, the business model and its success revolves around the potency of their recommendations. In fact, Netflix even offered a million dollars in 2009 to anyone who could improve its system by 10%.

In this tutorial, you will see how to build a basic model of simple as well as content-based recommender systems. While these models will be nowhere close to the industry standard in terms of complexity, quality or accuracy, it will help you to get started with building more complex models that produce even better results.

But what are these recommender systems?

Broadly, recommender systems can be classified into 3 types:

- Simple recommenders: offer generalized recommendations to every user, based on movie popularity and/or genre. The basic idea behind this system is that movies that are more popular and critically acclaimed will have a higher probability of being liked by the average audience. IMDB Top 250 is an example of this system.
- Content-based recommenders: suggest similar items based on a particular item. This system uses item metadata, such as genre, director, description, actors, etc. for movies, to make these recommendations. The general idea behind these recommender systems is that if a person liked a particular item, he or she will also like an item that is similar to it.
- Collaborative filtering engines: these systems try to predict the rating or preference that a user would give an item-based on past ratings and preferences of other users. Collaborative filters do not require item metadata like its content-based counterparts.

## Simple Recommenders

As described in the previous section, simple recommenders are basic systems that recommends the top items based on a certain metric or score. In this section, you will build a simplified clone of IMDB Top 250 Movies using metadata collected from IMDB.

The following are the steps involved:

- Decide on the metric or score to rate movies on.
- Calculate the score for every movie.
- Sort the movies based on the score and output the top results.
Before you perform any of the above steps, load your movies metadata dataset into a pandas DataFrame:

In [ ]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('./data/movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

One of the most basic metrics you can think of is the **rating**. However, using this metric has a few *caveats*. For one, it does not take into consideration the **popularity** (which is very important) of a movie. Therefore, a movie with a rating of `9` from `10` voters will be considered `'better'` than a movie with a rating of `8.9` from `10,000` voters.

On a related note, this metric will also tend to favor movies with smaller number of voters with skewed and/or extremely high ratings. As the number of voters increase, the rating of a movie *regularizes* and approaches towards a value that is reflective of the movie's quality. It is more difficult to discern the quality of a movie with **extremely** few voters.

Taking these shortcomings into consideration, it is necessary that you come up with a **weighted rating** that takes into account the average rating and the number of votes it has garnered. Such a system will make sure that a movie with a `9` rating from `100,000` voters gets a (far) **higher** score than a YouTube Web Series with the same rating but a few hundred voters.

Since you are trying to build a clone of `IMDB's Top 250`, you will use its weighted rating formula as your metric/score. Mathematically, $WR$ is represented as follows:

$$Weighted Rating (WR) = (\frac{v}{(v+m)}*R) + (\frac{m}{(v+m)}*C)$$

where,

- $v$ is the number of votes for the movie;
- $m$ is the minimum votes required to be listed in the chart;
- $R$ is the average rating of the movie; And
- $C$ is the mean vote across the whole data set.

You already have the values to $v$ (`vote_count`) and $R$ (`vote_average`) for each movie in the dataset. It is also possible to directly calculate $C$ from this data.

What you need to determine is an appropriate value for $m$, the minimum votes required to be listed in the chart. There is no right value for $m$. You can view it as a ***preliminary negative filter that ignores movies which have less than a certain number of votes***. The selectivity of your filter is up to your discretion (usually done in trial and error).

In this case, you will use the `90th` percentile as your cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least `90%` of the movies in the list. (On the other hand, if you had chosen the `75th` percentile, you would have considered the top `25%` of the movies in terms of the number of votes garnered. As the percentile decreases, the number of movies considered increases. Feel free to play with this value and observe the changes in your final chart).

As a first step, let's calculate the value of $C$, the mean rating across all movies:

In [ ]:
# Calculate C
C = metadata['vote_average'].mean()
print(C)

The average rating of a movie on IMDB is around `5.6`, on a scale of `10`.

Next, let's calculate the number of votes, $m$, received by a movie in the `90th` percentile. `Pandas` makes this task extremely trivial using the `.quantile()` method of a pandas Series:

In [ ]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

Next, you can filter the movies that qualify for the chart, based on their vote counts:

In [ ]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

#### YOUR TURN HERE

Answer this question: how many movie have reviews more than 90% of the movies in the dataset?

Double-click and type your answer here.

You use the `.copy()` method to ensure that the new `q_movies` DataFrame created is independent of your original metadata DataFrame. In other words, any changes made to the `q_movies` DataFrame does not affect the metadata.

You see that there are 4555 movies which qualify to be in this list. Now, you need to calculate your metric for each qualified movie. To do this, you will define a function, `weighted_rating()` and define a new feature `score`, of which you'll calculate the value by applying this function to your DataFrame of qualified movies:

In [ ]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

Finally, let's sort the DataFrame based on the `score` feature and output the title, vote count, vote average and weighted rating or score of the **top** 15 movies.

In [ ]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

You see that the chart has a lot of movies in common with the IMDB Top 250 chart: for example, your top two movies, "Shawshank Redemption" and "The Godfather", are the same as IMDB. In other words, if anyone search for any movie, these two should be the top 2 being recommended to them.

#### YOUR TURN HERE
Please try to tweak the $m$ parameter (originally set at `90th percentile`) to a `75th percentile`, and repeat all the steps to calculate the rating of movies.

In [ ]:
### Define a new `m1` parameter

### Filter out all qualified movies into a new DataFrame `my_movies`

### apply `weighted_rating()` to `my_movies` to calculate the new `score`

###Sort movies based on score calculated above

###Print the top 15 movies


**QUESTION:** Are the results different? If so, why?

**ANSWER**: type your answer here

## Content-Based Recommender in Python
### Plot Description Based Recommender
In this section, you will try to build a system that recommends movies that are similar to a **particular** movie. More specifically, you will compute `pairwise similarity scores` for all movies based on their plot descriptions and recommend movies based on that similarity score. You can see the improvement from the simple recommender system we built above

The plot description is available to you as the overview feature in your metadata dataset. Let's inspect the plots of a few movies:

In [ ]:
#Print plot overviews of the first 5 movies.
metadata['overview'].head()

In its current form, it is not possible to compute the similarity between any two overviews. To do this, you need to compute the word vectors of each overview or document, as it will be called from now on.

You will compute **Term Frequency-Inverse Document Frequency (TF-IDF)** vectors for each document. This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each column represents a movie, as before. Please note that this is an important step in **text analytics**.

In its essence, the TF-IDF score is the frequency of a word occurring in a document, down-weighted by the number of documents in which it occurs. This is done to reduce the importance of words that occur frequently in plot overviews and therefore (e.g. movie, actor - since they have little value in comparing movies), their significance in computing the final similarity score.

Fortunately, scikit-learn gives you a built-in `TfIdfVectorizer` class that produces the TF-IDF matrix in a couple of lines.

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')
my_metadata = metadata['overview'][:10000]

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(my_metadata)

#Output the shape of tfidf_matrix
tfidf_matrix.shape

You see that about `75,000` different words were used to describe over `45,000` movies in your dataset.

With this matrix in hand, you can now compute a similarity score. There are several candidates for this; such as the `euclidean`, the `Pearson` and the `cosine` similarity scores. Again, there is no right answer to which score is the best. Different scores work well in different scenarios and it is often a good idea to experiment with different metrics.

You will be using the `cosine similarity` to calculate a numeric quantity that denotes the similarity between two movies. You use the cosine similarity score since it is independent of magnitude and is relatively easy and fast to calculate (especially when used in conjunction with TF-IDF scores, which will be explained later). Mathematically, it is defined as follows: 

$$cosine(x,y) = \frac{x \cdot y^T}{||x||\cdot ||y||}$$

Since you have used the TF-IDF vectorizer, calculating the dot product will directly give you the cosine similarity score. Therefore, you will use sklearn's `linear_kernel()` instead of `cosine_similarities()` since it is faster.

#### DO IT YOURSELF
You can try to implement it using the `cosine_similarities()` function yourselves.

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

You're going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies. Firstly, for this, you need a reverse mapping of movie titles and DataFrame indices. In other words, you need a mechanism to identify the index of a movie in your `metadata` DataFrame, given its title.

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()
indices[:5]

You are now in a good position to define your recommendation function. These are the following steps you'll follow:

- Get the index of the movie given its title.
- Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position and the second is the similarity score.
- Sort the aforementioned list of tuples based on the similarity scores; that is, the second element.
- Get the top 10 elements of this list. Ignore the first element as it refers to self (the movie most similar to a particular movie is the movie itself).
- Return the titles corresponding to the indices of the top elements.

**NOTE**: Above are the programming logic for a function - when you are programming, the hardest part is to come up with the programming logic - I would suggest you study the programming logic above, and cross-compare with the function below to get an idea. Give it some practice so that you can write your own function easily.

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

Now we can test our recommender system using a few movies.

Let's first test it with `The Dark Knight Rises`:

In [ ]:
get_recommendations('Jumanji')

Anyone into classics? Let's test it with `The Godfather`:

In [ ]:
get_recommendations('The Godfather')

You see that, while your system has done a decent job of finding movies with similar plot descriptions, the quality of recommendations is not that great. "The Dark Knight Rises" returns all Batman movies while it more likely that the people who liked that movie are more inclined to enjoy other Christopher Nolan movies. This is something that cannot be captured by your present system.

### Credits, Genres and Keywords Based Recommender
It goes without saying that the quality of your recommender would be increased with the usage of better metadata. That is exactly what you are going to do in this section. You are going to build a recommender based on the following metadata: the 3 top actors, the director, related genres and the movie plot keywords.

The keywords, cast and crew data is not available in your current dataset so the first step would be to load and merge them into your main DataFrame.

In [ ]:
# Load keywords and credits
credits = pd.read_csv('./data/credits.csv')
keywords = pd.read_csv('./data/keywords.csv')

# Remove rows with bad IDs.
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [ ]:
# Print the first two movies of your newly merged metadata
metadata.head(2)


From your new features, cast, crew and keywords, you need to extract the three most important actors, the director and the keywords associated with that movie. Right now, your data is present in the form of "stringified" lists. You need to convert them into a form that is usable for you.

In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)


Next, you write functions that will help you to extract the required information from each feature. First, you'll import the NumPy package to get access to its `NaN` constant. Next, you can use it to write the `get_director()` function:

In [ ]:
# Import Numpy 
import numpy as np

In [ ]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

Please note how if-statement is written in functions - this is the most pythonic way.

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)


In [ ]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

The next step would be to convert the names and keyword instances into lowercase and strip all the spaces between them. This is done so that your vectorizer doesn't count the Johnny of "Johnny Depp" and "Johnny Galecki" as the same. After this processing step, the aforementioned actors will be represented as "johnnydepp" and "johnnygalecki" and will be distinct to your vectorizer.

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

You are now in a position to create your "metadata soup", which is a string that contains all the metadata that you want to feed to your vectorizer (namely actors, director and keywords).

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

The next steps are the same as what you did with your plot description based recommender. One important difference is that you use the `CountVectorizer()` instead of TF-IDF. This is because you do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense.

In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

You can now reuse your `get_recommendations()` function by passing in the new cosine_sim2 matrix as your second argument.

**NOTE**: in the `get_recommendations()` function we defined a default value for the `cosine_sim` parameter; but you can always update it by passing a new value to it. This is common practice in the field.

Let's try the same two examples we used above, and compare results.

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
get_recommendations('The Godfather', cosine_sim2)

You see that your recommender has been successful in capturing more information due to more metadata and has given you (arguably) better recommendations. There are, of course, numerous ways of playing with this system in order to improve recommendations.

#### Some suggestions:

- Introduce a popularity filter: this recommender would take the list of the 30 most similar movies, calculate the weighted ratings (using the IMDB formula from above), sort movies based on this rating and return the top 10 movies.
- Other crew members: other crew member names, such as screenwriters and producers, could also be included.
- Increasing weight of the director: to give more weight to the director, he or she could be mentioned multiple times in the soup to increase the similarity scores of movies with the same director.

You can find these ideas implemented in [this notebook](https://github.com/rounakbanik/movies/blob/master/movies_recommender.ipynb).



## Collaborative Filtering with Python
In this tutorial, you have learnt how to build your very own **Simple** and **Content Based** Movie Recommender Systems. There is also another extremely popular type of recommender known as **collaborative filters**.

Collaborative filters can further be classified into two types:

- User-based Filtering: these systems recommend products to a user that similar users have liked. For example, let's say Alice and Bob have a similar interest in books (that is, they largely like and dislike the same books). Now, let's say a new book has been launched into the market and Alice has read and loved it. It is therefore, highly likely that Bob will like it too and therefore, the system recommends this book to Bob.

- Item-based Filtering: these systems are extremely similar to the content recommendation engine that you built. These systems identify similar items based on how people have rated it in the past. For example, if Alice, Bob and Eve have given 5 stars to The Lord of the Rings and The Hobbit, the system identifies the items as similar. Therefore, if someone buys The Lord of the Rings, the system also recommends The Hobbit to him or her.

You will not be building these systems in this tutorial but you are already familiar with most of the ideas required to do so. A good place to start with collaborative filters is by examining the MovieLens dataset, which can be found [here](https://grouplens.org/datasets/movielens/).

## Conclusion
In this tutorial, you have covered how to build simple as well as content-based recommenders. Hopefully, you are now in a good position to make improvements on the basic systems you built and experiment with other kinds of recommenders (such as collaborative filters). I hope you had as much fun reading this as I had writing. Happy recommending!